# Experiment 003: Local Search Optimization

Building on grid placement (88.33), adding:
1. N=1 at 45° (quick win - 0.34 points)
2. Rotation tightening for all configs
3. Local search: move trees toward center
4. Fractional translation: fine-tune positions

In [1]:
import math
import numpy as np
import pandas as pd
from decimal import Decimal, getcontext
from shapely import affinity
from shapely.geometry import Polygon
from shapely.ops import unary_union
from shapely.strtree import STRtree
from scipy.spatial import ConvexHull
from scipy.optimize import minimize_scalar
import time
from numba import njit

# Set precision for Decimal
getcontext().prec = 25
scale_factor = Decimal('1e15')
SCALE = float(scale_factor)

print("Libraries loaded successfully")

Libraries loaded successfully


In [2]:
# Tree polygon template (numba-compatible)
@njit
def make_polygon_template():
    """Create the tree polygon template vertices."""
    tw = 0.15  # trunk width
    th = 0.2   # trunk height
    bw = 0.7   # base width
    mw = 0.4   # mid width
    ow = 0.25  # top width
    tip = 0.8
    t1 = 0.5
    t2 = 0.25
    base = 0.0
    tbot = -th
    
    x = np.array([0, ow/2, ow/4, mw/2, mw/4, bw/2, tw/2, tw/2, -tw/2, -tw/2, -bw/2, -mw/4, -mw/2, -ow/4, -ow/2], np.float64)
    y = np.array([tip, t1, t1, t2, t2, base, base, tbot, tbot, base, base, t2, t2, t1, t1], np.float64)
    return x, y

TX, TY = make_polygon_template()
print(f"Tree template: {len(TX)} vertices")

Tree template: 15 vertices


In [3]:
@njit
def rotate_point(x, y, angle_rad):
    """Rotate a point around origin."""
    c = math.cos(angle_rad)
    s = math.sin(angle_rad)
    return c * x - s * y, s * x + c * y

@njit
def get_tree_vertices(cx, cy, deg, tx, ty):
    """Get rotated and translated tree vertices."""
    n = len(tx)
    vx = np.empty(n, np.float64)
    vy = np.empty(n, np.float64)
    angle_rad = deg * math.pi / 180.0
    c = math.cos(angle_rad)
    s = math.sin(angle_rad)
    for i in range(n):
        vx[i] = c * tx[i] - s * ty[i] + cx
        vy[i] = s * tx[i] + c * ty[i] + cy
    return vx, vy

@njit
def get_bounding_box(xs, ys, degs, tx, ty):
    """Calculate bounding box for a group of trees."""
    n = len(xs)
    V = len(tx)
    mnx = 1e300
    mny = 1e300
    mxx = -1e300
    mxy = -1e300
    
    for i in range(n):
        r = degs[i] * math.pi / 180.0
        c = math.cos(r)
        s = math.sin(r)
        xi = xs[i]
        yi = ys[i]
        for j in range(V):
            X = c * tx[j] - s * ty[j] + xi
            Y = s * tx[j] + c * ty[j] + yi
            if X < mnx: mnx = X
            if X > mxx: mxx = X
            if Y < mny: mny = Y
            if Y > mxy: mxy = Y
    
    return mnx, mny, mxx, mxy

@njit
def get_side_length_fast(xs, ys, degs, tx, ty):
    """Calculate side length of bounding square."""
    mnx, mny, mxx, mxy = get_bounding_box(xs, ys, degs, tx, ty)
    return max(mxx - mnx, mxy - mny)

@njit
def score_group(xs, ys, degs, tx, ty):
    """Calculate score (side^2 / n) for a group."""
    n = len(xs)
    side = get_side_length_fast(xs, ys, degs, tx, ty)
    return side * side / n

print("Fast numba functions defined")

Fast numba functions defined


In [4]:
class ChristmasTree:
    """Represents a single, rotatable Christmas tree."""

    def __init__(self, center_x='0', center_y='0', angle='0'):
        self.center_x = Decimal(str(center_x))
        self.center_y = Decimal(str(center_y))
        self.angle = Decimal(str(angle))
        self._build_polygon()
    
    def _build_polygon(self):
        trunk_w = Decimal('0.15')
        trunk_h = Decimal('0.2')
        base_w = Decimal('0.7')
        mid_w = Decimal('0.4')
        top_w = Decimal('0.25')
        tip_y = Decimal('0.8')
        tier_1_y = Decimal('0.5')
        tier_2_y = Decimal('0.25')
        base_y = Decimal('0.0')
        trunk_bottom_y = -trunk_h

        initial_polygon = Polygon([
            (Decimal('0.0') * scale_factor, tip_y * scale_factor),
            (top_w / Decimal('2') * scale_factor, tier_1_y * scale_factor),
            (top_w / Decimal('4') * scale_factor, tier_1_y * scale_factor),
            (mid_w / Decimal('2') * scale_factor, tier_2_y * scale_factor),
            (mid_w / Decimal('4') * scale_factor, tier_2_y * scale_factor),
            (base_w / Decimal('2') * scale_factor, base_y * scale_factor),
            (trunk_w / Decimal('2') * scale_factor, base_y * scale_factor),
            (trunk_w / Decimal('2') * scale_factor, trunk_bottom_y * scale_factor),
            (-(trunk_w / Decimal('2')) * scale_factor, trunk_bottom_y * scale_factor),
            (-(trunk_w / Decimal('2')) * scale_factor, base_y * scale_factor),
            (-(base_w / Decimal('2')) * scale_factor, base_y * scale_factor),
            (-(mid_w / Decimal('4')) * scale_factor, tier_2_y * scale_factor),
            (-(mid_w / Decimal('2')) * scale_factor, tier_2_y * scale_factor),
            (-(top_w / Decimal('4')) * scale_factor, tier_1_y * scale_factor),
            (-(top_w / Decimal('2')) * scale_factor, tier_1_y * scale_factor),
        ])
        rotated = affinity.rotate(initial_polygon, float(self.angle), origin=(0, 0))
        self.polygon = affinity.translate(rotated,
                                          xoff=float(self.center_x * scale_factor),
                                          yoff=float(self.center_y * scale_factor))
    
    def update_position(self, new_x, new_y, new_angle=None):
        self.center_x = Decimal(str(new_x))
        self.center_y = Decimal(str(new_y))
        if new_angle is not None:
            self.angle = Decimal(str(new_angle))
        self._build_polygon()

print("ChristmasTree class defined")

ChristmasTree class defined


In [5]:
def has_overlap_strict(trees):
    """Check if any two trees overlap using STRICT criteria."""
    if len(trees) <= 1:
        return False, []
    
    polygons = [t.polygon for t in trees]
    tree_index = STRtree(polygons)
    overlap_details = []
    
    for i, poly in enumerate(polygons):
        indices = tree_index.query(poly)
        for idx in indices:
            if idx <= i:
                continue
            if poly.intersects(polygons[idx]):
                intersection = poly.intersection(polygons[idx])
                area = intersection.area / (SCALE ** 2)
                if area > 1e-15:
                    overlap_details.append((i, idx, area))
    
    return len(overlap_details) > 0, overlap_details

def has_overlap_single(trees, tree_idx):
    """Check if a single tree overlaps with any other tree."""
    if len(trees) <= 1:
        return False
    
    polygons = [t.polygon for t in trees]
    target_poly = polygons[tree_idx]
    tree_index = STRtree(polygons)
    
    indices = tree_index.query(target_poly)
    for idx in indices:
        if idx == tree_idx:
            continue
        if target_poly.intersects(polygons[idx]):
            intersection = target_poly.intersection(polygons[idx])
            area = intersection.area / (SCALE ** 2)
            if area > 1e-15:
                return True
    return False

print("Overlap detection functions defined")

Overlap detection functions defined


In [6]:
def find_best_trees_grid(n):
    """Find the best grid-based placement for n trees."""
    best_score = float('inf')
    best_trees = None
    
    for n_even in range(1, n + 1):
        for n_odd in [n_even, n_even - 1]:
            if n_odd < 0:
                continue
                
            all_trees = []
            rest = n
            r = 0
            
            while rest > 0:
                m = min(rest, n_even if r % 2 == 0 else n_odd)
                if m <= 0:
                    break
                rest -= m
                
                angle = 0 if r % 2 == 0 else 180
                x_offset = Decimal('0') if r % 2 == 0 else Decimal('0.35')
                
                if r % 2 == 0:
                    y = Decimal(str(r // 2)) * Decimal('1.0')
                else:
                    y = Decimal('0.8') + Decimal(str((r - 1) // 2)) * Decimal('1.0')
                
                row_trees = [
                    ChristmasTree(
                        center_x=Decimal('0.7') * Decimal(str(i)) + x_offset,
                        center_y=y,
                        angle=angle
                    )
                    for i in range(m)
                ]
                all_trees.extend(row_trees)
                r += 1
            
            if len(all_trees) != n:
                continue
            
            # Calculate score using fast method
            xs = np.array([float(t.center_x) for t in all_trees])
            ys = np.array([float(t.center_y) for t in all_trees])
            degs = np.array([float(t.angle) for t in all_trees])
            score = score_group(xs, ys, degs, TX, TY)
            
            if score < best_score:
                best_score = score
                best_trees = all_trees
    
    return best_score, best_trees

print("Grid placement function defined")

Grid placement function defined


In [7]:
def rotation_tightening(trees):
    """Rotate entire configuration to minimize bounding box."""
    if len(trees) <= 1:
        return trees, 0.0
    
    # Get all vertices
    all_points = []
    for tree in trees:
        coords = list(tree.polygon.exterior.coords)
        for x, y in coords:
            all_points.append([x / SCALE, y / SCALE])
    
    points_np = np.array(all_points)
    
    # Use convex hull for efficiency
    try:
        hull_points = points_np[ConvexHull(points_np).vertices]
    except:
        hull_points = points_np
    
    def bbox_side_at_angle(angle_deg):
        angle_rad = np.radians(angle_deg)
        c, s = np.cos(angle_rad), np.sin(angle_rad)
        rot_matrix_T = np.array([[c, s], [-s, c]])
        rotated_points = hull_points.dot(rot_matrix_T)
        min_xy = np.min(rotated_points, axis=0)
        max_xy = np.max(rotated_points, axis=0)
        return max(max_xy[0] - min_xy[0], max_xy[1] - min_xy[1])
    
    initial_side = bbox_side_at_angle(0)
    
    res = minimize_scalar(bbox_side_at_angle, bounds=(0.001, 89.999), method='bounded')
    best_angle = res.x
    best_side = res.fun
    
    if best_side < initial_side - 1e-9:
        # Apply rotation to all trees
        # Find center of configuration
        xs = np.array([float(t.center_x) for t in trees])
        ys = np.array([float(t.center_y) for t in trees])
        cx, cy = xs.mean(), ys.mean()
        
        angle_rad = np.radians(best_angle)
        c, s = np.cos(angle_rad), np.sin(angle_rad)
        
        for tree in trees:
            # Rotate position around center
            dx = float(tree.center_x) - cx
            dy = float(tree.center_y) - cy
            new_x = c * dx - s * dy + cx
            new_y = s * dx + c * dy + cy
            new_angle = float(tree.angle) + best_angle
            tree.update_position(new_x, new_y, new_angle)
        
        return trees, best_angle
    
    return trees, 0.0

print("Rotation tightening function defined")

Rotation tightening function defined


In [8]:
def local_search(trees, max_iters=100):
    """Move trees toward center to reduce bounding box."""
    if len(trees) <= 1:
        return trees
    
    # Get current state as numpy arrays for fast computation
    xs = np.array([float(t.center_x) for t in trees])
    ys = np.array([float(t.center_y) for t in trees])
    degs = np.array([float(t.angle) for t in trees])
    
    best_side = get_side_length_fast(xs, ys, degs, TX, TY)
    steps = [0.02, 0.01, 0.005, 0.002]
    
    for iteration in range(max_iters):
        improved = False
        
        # Calculate centroid
        cx, cy = xs.mean(), ys.mean()
        
        for i in range(len(trees)):
            # Direction toward center
            dx = cx - xs[i]
            dy = cy - ys[i]
            dist = math.sqrt(dx * dx + dy * dy)
            
            if dist < 1e-6:
                continue
            
            dx /= dist
            dy /= dist
            
            for step in steps:
                # Save original
                orig_x, orig_y = xs[i], ys[i]
                
                # Try new position
                xs[i] = orig_x + dx * step
                ys[i] = orig_y + dy * step
                trees[i].update_position(xs[i], ys[i])
                
                # Check overlap
                if not has_overlap_single(trees, i):
                    new_side = get_side_length_fast(xs, ys, degs, TX, TY)
                    if new_side < best_side - 1e-12:
                        best_side = new_side
                        improved = True
                        break  # Keep this improvement
                    else:
                        # Revert
                        xs[i] = orig_x
                        ys[i] = orig_y
                        trees[i].update_position(xs[i], ys[i])
                else:
                    # Revert
                    xs[i] = orig_x
                    ys[i] = orig_y
                    trees[i].update_position(xs[i], ys[i])
        
        if not improved:
            break
    
    return trees

print("Local search function defined")

Local search function defined


In [9]:
def fractional_translation(trees, max_iters=100):
    """Fine-tune positions with small fractional moves in 8 directions."""
    if len(trees) <= 1:
        return trees
    
    xs = np.array([float(t.center_x) for t in trees])
    ys = np.array([float(t.center_y) for t in trees])
    degs = np.array([float(t.angle) for t in trees])
    
    best_side = get_side_length_fast(xs, ys, degs, TX, TY)
    
    frac_steps = [0.001, 0.0005, 0.0002, 0.0001]
    directions = [(0, 1), (0, -1), (1, 0), (-1, 0), (1, 1), (1, -1), (-1, 1), (-1, -1)]
    
    for iteration in range(max_iters):
        improved = False
        
        for i in range(len(trees)):
            for step in frac_steps:
                for dx, dy in directions:
                    orig_x, orig_y = xs[i], ys[i]
                    
                    xs[i] = orig_x + dx * step
                    ys[i] = orig_y + dy * step
                    trees[i].update_position(xs[i], ys[i])
                    
                    if not has_overlap_single(trees, i):
                        new_side = get_side_length_fast(xs, ys, degs, TX, TY)
                        if new_side < best_side - 1e-12:
                            best_side = new_side
                            improved = True
                        else:
                            xs[i] = orig_x
                            ys[i] = orig_y
                            trees[i].update_position(xs[i], ys[i])
                    else:
                        xs[i] = orig_x
                        ys[i] = orig_y
                        trees[i].update_position(xs[i], ys[i])
        
        if not improved:
            break
    
    return trees

print("Fractional translation function defined")

Fractional translation function defined


In [10]:
# Generate optimized solutions for all N from 1 to 200
start_time = time.time()

solutions = {}  # n -> trees
scores = {}  # n -> score

for n in range(1, 201):
    # Special case: N=1 at 45 degrees
    if n == 1:
        trees = [ChristmasTree(center_x='0', center_y='0', angle='45')]
    else:
        # Grid placement
        _, trees = find_best_trees_grid(n)
        
        # Rotation tightening
        trees, rot_angle = rotation_tightening(trees)
        
        # Local search
        trees = local_search(trees, max_iters=50)
        
        # Fractional translation
        trees = fractional_translation(trees, max_iters=30)
    
    solutions[n] = trees
    
    # Calculate score
    xs = np.array([float(t.center_x) for t in trees])
    ys = np.array([float(t.center_y) for t in trees])
    degs = np.array([float(t.angle) for t in trees])
    scores[n] = score_group(xs, ys, degs, TX, TY)
    
    if n % 20 == 0:
        elapsed = time.time() - start_time
        print(f"N={n:3d}: score={scores[n]:.6f}, elapsed={elapsed:.1f}s")

total_time = time.time() - start_time
print(f"\nTotal time: {total_time:.1f}s")

N= 20: score=0.496125, elapsed=4.5s


N= 40: score=0.441000, elapsed=13.9s


N= 60: score=0.450667, elapsed=33.1s


N= 80: score=0.442531, elapsed=65.1s


N=100: score=0.396900, elapsed=112.9s


N=120: score=0.408333, elapsed=178.9s


N=140: score=0.385875, elapsed=266.4s


N=160: score=0.405016, elapsed=377.3s


N=180: score=0.392000, elapsed=514.4s


N=200: score=0.405000, elapsed=681.9s

Total time: 681.9s


In [ ]:
# Calculate total score
total_score = sum(scores[n] for n in range(1, 201))
print(f"Total score: {total_score:.6f}")
print(f"Target score: 68.947559")
print(f"Gap: {total_score - 68.947559:.6f}")
print(f"\nImprovement from grid (88.33): {88.33 - total_score:.6f} points")

In [ ]:
# Validate all configurations
print("Validating all configurations...")
overlap_issues = []

for n in range(1, 201):
    trees = solutions[n]
    has_overlap, details = has_overlap_strict(trees)
    if has_overlap:
        overlap_issues.append((n, details))
        print(f"  N={n}: OVERLAP! {details[:3]}...")

if overlap_issues:
    print(f"\nWARNING: Overlaps found in {len(overlap_issues)} groups!")
else:
    print("\n✓ All configurations valid - no overlaps detected!")

In [ ]:
# Create submission
def to_str(x):
    return f"s{round(float(x), 6)}"

rows = []
for n in range(1, 201):
    trees = solutions[n]
    assert len(trees) == n, f"Expected {n} trees, got {len(trees)}"
    for i_t, tree in enumerate(trees):
        rows.append({
            "id": f"{n:03d}_{i_t}",
            "x": to_str(tree.center_x),
            "y": to_str(tree.center_y),
            "deg": to_str(tree.angle),
        })

submission = pd.DataFrame(rows)
print(f"Submission shape: {submission.shape}")
print(submission.head(10))

In [ ]:
# Save submission
submission.to_csv('/home/submission/submission.csv', index=False)
submission.to_csv('/home/code/experiments/003_local_search/submission.csv', index=False)
print("Submission saved!")
print(f"\nFinal score: {total_score:.6f}")